# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [17]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [18]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [19]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [20]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [21]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [22]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Queries to ask the following three questions of the data

### QUERY 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Description: Here the Primary Key has two fields: session_id is the partition key, and item_in_session is clustering key. Partitioning is done by session_id and within that partition, rows are ordered by the item_in_session.

In [61]:
drop_it = "drop table session_id_item"
try:
    session.execute(drop_it)
except Exception as e:
    print(e)

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})


In [25]:
query = "CREATE TABLE IF NOT EXISTS session_id_item"
query = query + "(session_id int, item_in_session int, artist text, song text, length float,  \
                  PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [28]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query =  "INSERT INTO session_id_item (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [30]:
import pandas as pd

query_1 = "SELECT artist, song, length from session_id_item WHERE session_id=338 AND item_in_session=4"

try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)

df = pd.DataFrame(list(rows))
print(df)

      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312



### QUERY 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Description: Here the Primary Key has three fields: user_id and session_id are partition keys so sessions from the same user are stored together, item_in_session is the clustering key. 

In [45]:
drop_it = "drop table user_session"
try:
    session.execute(drop_it)
except Exception as e:
    print(e)

In [46]:

query = "CREATE TABLE IF NOT EXISTS user_session"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, \
                  first_name text, last_name text, \
                  PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [47]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query =  "INSERT INTO user_session (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [50]:
query_2 = "SELECT artist, song, first_name, last_name FROM user_session WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)

df = pd.DataFrame(list(rows))
print(df)

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


### QUERY 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Description: The primary key has two fields: song_title and user_id. So we can identify every record uniquely.


In [54]:
drop_it = "drop table song_user"
try:
    session.execute(drop_it)
except Exception as e:
    print(e)

In [55]:
query = "CREATE TABLE IF NOT EXISTS song_user"
query = query + "(song text, user_id int, first_name text, last_name text, \
                  PRIMARY KEY (song, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [57]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query =  "INSERT INTO song_user (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [58]:
query_2 = "select first_name, last_name from song_user WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
print(df)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [59]:
query= "drop table session_id_item"
query2= "drop table user_session"
query3= "drop table song_user"

try:
    rows = session.execute(query)
    rows = session.execute(query2)
    rows = session.execute(query3)

except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [60]:
session.shutdown()
cluster.shutdown()